In [11]:
import numpy as np


class Decision_Tree_Regression(object):
    def __init__(self, max_depth=5, sample_split_thres=2):
        self.sample_split_thres = sample_split_thres
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self.__construct(X, y)

    def __construct(self, X, y, curr_depth = 0):
        sample_num = np.shape(X)[0]
        if sample_num >= self.sample_split_thres and curr_depth <= self.max_depth:
            best_feature, best_thres = self.__find_best_split(X, y)
            left_samples, right_samples = self.__split_left_right(X[:, best_feature], best_thres)
            left = self.__construct(X[left_samples, :], y[left_samples], curr_depth + 1)
            right = self.__construct(X[right_samples, :], y[right_samples], curr_depth + 1)
            return Node(best_feature, best_thres, left, right) # internal node -> value is None
        leaf_val = np.mean(y)  # mean of the target values
        return Node(value = leaf_val) # leaf node -> other attributes are None

    def __find_best_split(self, X, y):
        feature_num = np.shape(X)[1]
        best_feature, best_thres = None, None
        min_impurity = float('inf') # we want to minimize the impurity
        for feature_ind in range(feature_num):
            thresholds = np.unique(X[:, feature_ind])
            for thres in thresholds:
                impurity = self.__calculate_impurity(y, thres, X[:, feature_ind])
                if impurity < min_impurity:
                    min_impurity = impurity
                    best_feature = feature_ind
                    best_thres = thres
        return best_feature, best_thres # these indicate the best way to split the data

    def __calculate_impurity(self, y, thres, feature):
        left_samples, right_samples = self.__split_left_right(feature, thres)
        if len(left_samples) == 0 or len(right_samples) == 0:
            return float('inf')
        left_labels = y[left_samples]
        right_labels = y[right_samples]
        p_left = len(left_labels) / len(y)  # proportion of the left subtree
        p_right = 1 - p_left  # proportion of the right subtree
        impurity = p_left * self.__calculate_gini(left_labels) + p_right * self.__calculate_gini(right_labels)
        return impurity

    def __calculate_gini(self, y):
        uniques, counts = np.unique(y, return_counts=True)
        p_classes = counts / len(y) # proportion of each class
        return 1 - np.sum(np.square(p_classes))  # gini impurity
        # gini := 1 - sigma_sum (p_i^2)

    def __split_left_right(self, feature, thres):
        # indices of the left subtree
        left_samples = np.argwhere(feature <= thres).flatten()
        # indices of the right subtree
        right_samples = np.argwhere(feature > thres).flatten()
        return left_samples, right_samples

    def predict(self, X):
        return np.array([self.__find_x_from_node(x, self.root) for x in X])

    def __find_x_from_node(self, x, node):
        if node.value is not None:  # leaf node
            return node.value  # return the value of the leaf node
        feature_value = x[node.feature_ind]  # get the value of the feature
        if feature_value <= node.thres:  # if <= thres, go left
            return self.__find_x_from_node(x, node.left)
        return self.__find_x_from_node(x, node.right)


class Node(object):
    def __init__(self, feature_ind=None, thres=None, left=None, right=None, value=None):
        self.feature_ind = feature_ind
        self.thres = thres
        self.left = left
        self.right = right
        self.value = value

    def __repr__(self):
        if self.value is None:
            return "Internal(feature_ind=%s, thres=%f)"%(self.feature_ind, self.thres)
        return "Leaf(value=%f)"%(self.value)


def mean_squared_error(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))


In [3]:
import numpy as np
path = 'data/'
X_train = np.load(path + 'X_train.npy')
y_train = np.load(path + 'y_train.npy')
X_val = np.load(path + 'X_val.npy')
y_val = np.load(path + 'y_val.npy')
X_test = np.load(path + 'X_test.npy')
y_test = np.load(path + 'y_test.npy')

In [13]:
for i in range(14, 15):
    model = Decision_Tree_Regression(max_depth=12, sample_split_thres=i)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    print("MSE:", mean_squared_error(y_val, y_pred))
    print("sample_split_thres:", i)

MSE: 54.007142432058295
sample_split_thres: 11
MSE: 54.341599203564314
sample_split_thres: 12
MSE: 54.28764317266041
sample_split_thres: 13
MSE: 54.23639863284722
sample_split_thres: 14


In [ ]:
MSE: 54.88012575771635
sample_split_thres: 8
MSE: 54.53741860584942
sample_split_thres: 9
MSE: 54.26272858630605
sample_split_thres: 10
MSE: 54.007142432058295
sample_split_thres: 11
MSE: 54.007142432058295
sample_split_thres: 11
MSE: 54.341599203564314
sample_split_thres: 12
MSE: 54.28764317266041
sample_split_thres: 13
MSE: 54.23639863284722
sample_split_thres: 14

In [7]:
min_error = float('inf')
best_depth = None
best_net = None
best_split = None

for i in range(10, 13):
    for j in range(2, 10):
        net = Decision_Tree_Regression(max_depth=i, sample_split_thres=j)
        net.fit(X_train, y_train)
        y_val_hat = net.predict(X_val)
        error = mean_squared_error(y_val, y_val_hat)
        print('depth: {}, split: {}, error: {}'.format(i, j, error))
        if error < min_error:
            min_error = error
            best_net = net
            best_depth = i
            best_split = j

depth: 10, split: 2, error: 56.78696963411451
depth: 10, split: 3, error: 56.79988706577163
depth: 10, split: 4, error: 56.30567087769273
depth: 10, split: 5, error: 56.26798248379765
depth: 10, split: 6, error: 56.31875922801334
depth: 10, split: 7, error: 56.52551989496126
depth: 10, split: 8, error: 56.50387365186174
depth: 10, split: 9, error: 56.42948998183488
depth: 11, split: 2, error: 57.04300139394967
depth: 11, split: 3, error: 56.98894406607251
depth: 11, split: 4, error: 56.1317550632788
depth: 11, split: 5, error: 56.04419688826381
depth: 11, split: 6, error: 55.947680315078784
depth: 11, split: 7, error: 56.19645383998316
depth: 11, split: 8, error: 56.153669585505305
depth: 11, split: 9, error: 56.08066101954237
depth: 12, split: 2, error: 57.13901215974242
depth: 12, split: 3, error: 57.0993010983182
depth: 12, split: 4, error: 56.12516216489861
depth: 12, split: 5, error: 55.77690033684333
depth: 12, split: 6, error: 55.64173109781663
depth: 12, split: 7, error: 55.868

In [ ]:
losses = []
losses.append([56.78696963411451, 56.79988706577163, 56.30567087769273, 56.26798248379765, 56.31875922801334, 56.52551989496126, 56.50387365186174, 56.42948998183488])
losses.append([57.04300139394967, 56.98894406607251, 56.1317550632788, 56.04419688826381, 55.947680315078784, 56.19645383998316, 56.153669585505305, 56.08066101954237])
losses.append([57.13901215974242, 57.0993010983182, 56.12516216489861, 55.77690033684333, 55.64173109781663, 55.86875157070448, 55.79222879860492, 55.721369012839155])
splits = [2, 3, 4, 5, 6, 7, 8, 9]

import matplotlib.pyplot as plt

plt.plot(splits, losses[0], label='depth=10')
plt.plot(splits, losses[1], label='depth=11')
plt.plot(splits, losses[2], label='depth=12')
plt.legend()
plt.xlabel('sample_split_thres')
plt.ylabel('MSE')
plt.show()

In [ ]:
10, 2, 56.78696963411451
10, 3, 56.79988706577163
10, 4, 56.30567087769273
10, 5, 56.26798248379765
10, 6, 56.31875922801334
10, 7, 56.52551989496126
10, 8, 56.50387365186174
10, 9, 56.42948998183488
11, 2, 57.04300139394967
11, 3, 56.98894406607251
11, 4, 56.1317550632788
11, 5, 56.04419688826381
11, 6, 55.947680315078784
11, 7, 56.19645383998316
11, 8, 56.153669585505305
11, 9, 56.08066101954237
12, 2, 57.13901215974242
12, 3, 57.0993010983182
12, 4, 56.12516216489861
12, 5, 55.77690033684333
12, 6, 55.64173109781663
12, 7, 55.86875157070448
12, 8, 55.79222879860492
12, 9, 55.721369012839155